## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [7]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.44 ms ± 97 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 34.3 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 27 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


# 3.2 a) Librería Cupy

In [5]:
import cupy as cp
import numpy as np
from cupyx.profiler import benchmark 

# 1. Definimos la función matemática (ufunc)
def grade2_ufunc(x, y, a, b, c):
    return a * x**2 + b * y + c

# Parámetros del ejercicio
size = 5_000_000
a, b, c = 3.5, 2.8, 10.0

print(f"--- Benchmarking CuPy (N={size}) ---\n")

# ==========================================
# CASO A: SIN COPIA (Datos creados en GPU)
# ==========================================
# Creamos los arrays directamente en la VRAM de la GPU
x_gpu = cp.random.rand(size, dtype=cp.float32)
y_gpu = cp.random.rand(size, dtype=cp.float32)

print("1. Medición SIN copia de datos (Datos ya en GPU):")
# benchmark ejecuta la función varias veces y devuelve estadísticas
result_no_copy = benchmark(grade2_ufunc, (x_gpu, y_gpu, a, b, c), n_repeat=10)
print(result_no_copy)


# ==========================================
# CASO B: CON COPIA (CPU -> GPU -> Cálculo)
# ==========================================
# Creamos los arrays en la RAM de la CPU (NumPy)
x_cpu = np.random.rand(size).astype(np.float32)
y_cpu = np.random.rand(size).astype(np.float32)

# Definimos una función envoltorio que incluya la transferencia explícita
# para que el benchmark mida también el tiempo de mover los datos.
def grade2_with_copy(x, y, a, b, c):
    x_d = cp.asarray(x) # Copia CPU -> GPU
    y_d = cp.asarray(y) # Copia CPU -> GPU
    return a * x_d**2 + b * y_d + c

print("\n2. Medición CON copia de datos (Incluye transferencia CPU->GPU):")
result_copy = benchmark(grade2_with_copy, (x_cpu, y_cpu, a, b, c), n_repeat=10)
print(result_copy)

--- Benchmarking CuPy (N=5000000) ---

1. Medición SIN copia de datos (Datos ya en GPU):
grade2_ufunc        :    CPU:   100.920 us   +/- 10.565 (min:    93.750 / max:   129.815) us     GPU-0:   900.435 us   +/-  4.134 (min:   896.896 / max:   912.384) us

2. Medición CON copia de datos (Incluye transferencia CPU->GPU):
grade2_with_copy    :    CPU:  3478.112 us   +/- 83.248 (min:  3372.638 / max:  3608.178) us     GPU-0:  5781.011 us   +/- 83.061 (min:  5664.768 / max:  5910.528) us


# 3.2. b) Librería Numba

In [1]:
from numba import vectorize, cuda, float32
import numpy as np
import cupy as cp
from cupyx.profiler import benchmark

# ---------------------------------------------------------
# 1. Definición del Kernel con Numba
# ---------------------------------------------------------
# Usamos el decorador @vectorize para crear una ufunc que corra en GPU (target='cuda').
# Firma: devuelve float32, recibe 5 argumentos float32.
@vectorize(['float32(float32, float32, float32, float32, float32)'], target='cuda')
def grade2_numba(x, y, a, b, c):
    return a * x**2 + b * y + c

# Parámetros iniciales
size = 5_000_000
a, b, c = 3.5, 2.8, 10.0

print(f"--- Benchmarking Numba CUDA (N={size}) ---\n")

# ---------------------------------------------------------
# 2. Preparación de datos (CPU y GPU)
# ---------------------------------------------------------
# Datos en Host (CPU) para el caso "CON COPIA"
x_cpu = np.random.rand(size).astype(np.float32)
y_cpu = np.random.rand(size).astype(np.float32)

# Datos en Device (GPU) para el caso "SIN COPIA"
# Copiamos manualmente fuera del bucle de medición
x_device = cuda.to_device(x_cpu)
y_device = cuda.to_device(y_cpu)

# ---------------------------------------------------------
# 3. Medición: Caso SIN COPIA (Datos residentes en VRAM)
# ---------------------------------------------------------
print("1. Medición SIN copia (usando arrays ya en GPU):")
# Al pasarle 'x_device' (que es un puntero a GPU), Numba detecta que no hace falta copiar.
# Importante: benchmark() ejecuta la función repetidas veces para sacar estadísticas.
res_no_copy = benchmark(grade2_numba, (x_device, y_device, a, b, c), n_repeat=10)
print(res_no_copy)

# ---------------------------------------------------------
# 4. Medición: Caso CON COPIA (Transferencia automática)
# ---------------------------------------------------------
print("\n2. Medición CON copia (pasando arrays de NumPy):")
# Al pasarle 'x_cpu' (NumPy), Numba hace automáticamente: Host -> Device -> Compute -> Host
res_with_copy = benchmark(grade2_numba, (x_cpu, y_cpu, a, b, c), n_repeat=10)
print(res_with_copy)

--- Benchmarking Numba CUDA (N=5000000) ---

1. Medición SIN copia (usando arrays ya en GPU):
grade2_numba        :    CPU:  1505.309 us   +/- 700.690 (min:   840.425 / max:  2399.791) us     GPU-0:  1641.728 us   +/- 596.791 (min:  1065.792 / max:  2408.448) us

2. Medición CON copia (pasando arrays de NumPy):
grade2_numba        :    CPU:  9168.472 us   +/- 1394.156 (min:  7259.503 / max: 11444.168) us     GPU-0:  9175.901 us   +/- 1395.124 (min:  7266.080 / max: 11455.168) us


# 3.2. c) 

1. La conclusión al comparar los casos "con Copia" y "sin Copia" en ambas librerías es el coste masivo de mover datos.

- Sin Copia (Datos en GPU): Cuando los datos ya residen en la memoria de la GPU (VRAM), el tiempo de ejecución es del orden de microsegundos (μs). Esto se debe a que la GPU tiene un ancho de banda de memoria extremadamente alto (cientos de GB/s) y miles de núcleos procesando en paralelo.

- Con Copia (CPU ↔ GPU): Al incluir la transferencia, el tiempo aumenta drásticamente (a milisegundos). Esto demuestra que el bus PCIe actúa como un cuello de botella significativo .

La estrategia óptima es mantener los datos en la GPU el mayor tiempo posible.

2. Análisis de CuPy: se comporta como un "espejo" de NumPy. Al usar cp.asarray, forzamos la copia explicita.

En el caso "sin Copia", CuPy es extremadamente rápido porque lanza kernels pre-compilados y optimizados para operaciones vectoriales básicas. Sin embargo, para fórmulas complejas, CuPy podría generar arrays temporales intermedios en la VRAM, lo cual consume más memoria y ancho de banda que un kernel fusionado.

3. Análisis de Numba: Numba @vectorize compila la función Python a un kernel CUDA nativo.

Numba es inteligente detectando el tipo de entrada. Si recibe arrays de NumPy, realiza la copia implícita (Host → Device → Host), penalizando el rendimiento igual que en el caso de CuPy con copia. Si recibe punteros de dispositivo (cuda.to_device), ejecuta a velocidad nativa.

A diferencia de CuPy (que llama a kernels separados para suma, multiplicación, potencia), Numba fusiona toda la operación a*x**2 + b*y + c en un solo kernel. Esto significa que cada elemento se lee de memoria una sola vez, se opera y se escribe una sola vez.

Teóricamente, Numba "Sin Copia" debería ser ligeramente más eficiente en uso de memoria y caché que CuPy para operaciones encadenadas, aunque ambos son órdenes de magnitud más rápidos que la CPU.